In [35]:
from utils.news import get_news, get_url_content
from utils.price import get_price
from utils.llms import generate_impact_summary, generate_overall_summary, get_trading_recommendation, summarize
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm

In [36]:
stock = "Nvidia"
symbol = "NVDA"  
date = "2024-08-14"
news_duration = 2
price_duration = 7
sector_info='''Nvidia operates in the semiconductor industry, specializing in the design and manufacture of graphics processing units (GPUs) for gaming, professional visualization, data centers, AI, and automotive applications. 
Known for its GeForce brand, Nvidia leads the gaming market while also providing high-performance solutions for AI and machine learning in data centers. 
Additionally, Nvidia's DRIVE platform supports autonomous vehicle technology, and its Jetson platform powers edge computing and IoT devices. 
The company's innovations position it at the forefront of emerging trends in AI, data processing, and autonomous driving, driving significant growth and industry influence.
'''

In [37]:
news_df = get_news(symbol, date, news_duration)

In [38]:
filtered_news_df = news_df[news_df['symbols'].apply(lambda x: symbol in x)]

In [32]:
url_list = filtered_news_df['url'].values.tolist()
topic_list=[]
content_list=[]
for url in tqdm(url_list):
    topic, content = get_url_content(url)    
    topic_list.append(topic)
    content_list.append(content) 

100%|███████████████████████████████████████████| 15/15 [00:15<00:00,  1.02s/it]


In [33]:
filtered_news_df['topic'] = topic_list
filtered_news_df['content'] = content_list

In [34]:
filtered_news_df.to_csv('data/news_content.csv',index=False)

In [3]:
filtered_news_df = pd.read_csv('data/news_content.csv')

In [4]:
# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
embeddings = model.encode(filtered_news_df.content.values)
print(embeddings.shape)

(13, 384)


In [6]:
embeddings_np = np.array(embeddings).astype('float32')

# Create a FAISS index
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add vectors to the index
index.add(embeddings_np)

# Save the index to a file
faiss.write_index(index, "data/news_content_index.faiss")

In [7]:
# # Load the FAISS index
# index = faiss.read_index("database/pdf_sections_index.faiss")

# # Load the embedding model
# model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
def search_faiss(query, k=5):
    query_vector = model.encode([query])[0].astype('float32')
    query_vector = np.expand_dims(query_vector, axis=0)
    distances, indices = index.search(query_vector, k)
    return distances, indices

In [9]:
# Example usage
query = "Which article would have the highest impact on nvidia stock price? Ticker: NVDA"

In [10]:
distances, indices = search_faiss(query)

print(f"Query: {query}")
print(f"Distances: {distances}")
print(f"Indices: {indices}")

Query: Which article would have the highest impact on nvidia stock price? Ticker: NVDA
Distances: [[0.67116094 0.9870448  1.0481676  1.0679644  1.137571  ]]
Indices: [[0 5 8 1 6]]


In [11]:
relevant_news_df = filtered_news_df[filtered_news_df.index.isin(indices[0])]

In [12]:
summary_list=[]
for i in tqdm(range(len(relevant_news_df))):
    topic = relevant_news_df.topic.values[i]
    content = relevant_news_df.content.values[i]
    summary = summarize(stock, topic, content)
    summary_list.append(summary)

  0%|                                                     | 0/5 [00:00<?, ?it/s]/Users/arnabchakraborty/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/arnabchakraborty/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
100%|█████████████████████████████████████████████| 5/5 [00:41<00:00,  8.36s/it]


In [13]:
relevant_news_df['summary'] = summary_list

/var/folders/04/3srmk7w95zx2fd802j47m0ww0000gn/T/ipykernel_97600/294216855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_news_df['summary'] = summary_list


In [17]:
print(summary_list[2])

Here is a concise summary of the article focusing on Nvidia (NVDA):

Nvidia shares fell over 5% in response to Advanced Micro Devices' (AMD) less-than-expected second-quarter guidance. This led to a decline in the broader chipmaking industry, with NVDA experiencing a significant price drop.


In [18]:
overall_summary = generate_overall_summary(stock, relevant_news_df)

In [20]:
print(overall_summary)

Based on the provided news items, here is a concise summary of the major key points that might impact Nvidia's stock price:

Positive Factors:

* The upcoming first-quarter earnings report, scheduled for May 22, has the potential to trigger Nvidia's next bullish phase. A positive report could push the stock to surpass its previous highs.
* Nvidia's strong moat due to its intellectual property (IP) for its Graphics Processing Units (GPUs), as mentioned in Topic 4.

Negative Factors:

* The recent rollercoaster ride of Nvidia's stock, with a 96% bull rally followed by a 13% dip, suggests that the company is consolidating between $756 and $974. A negative earnings report could lead to a downturn.
* Nvidia's shares fell over 5% in response to Advanced Micro Devices' (AMD) less-than-expected second-quarter guidance (Topic 2).
* The general sentiment towards Nvidia is cautious, with money flows being negative (Topic 4).

Recurring Themes/Trends:

* The importance of Nvidia's earnings report 

In [21]:
price = get_price(symbol, date, price_duration)
table_string = price.to_string(index=False)

[*********************100%%**********************]  1 of 1 completed


In [25]:
trade_recommendation = get_trading_recommendation(sector_info, date, table_string, overall_summary)

In [26]:
print(trade_recommendation)

Decision: Hold
Reasoning:
1. The recent stock price momentum of Nvidia is volatile, with a sharp increase followed by a dip. This suggests that the market is still adjusting to the company's performance and may not be ready for another significant upward move.
2. While the upcoming earnings report has the potential to trigger a bullish phase, it's crucial to exercise caution given the recent volatility and negative market trends.
3. The general sentiment towards Nvidia is cautious, indicating that investors are advising patience and vigilance in monitoring key support and resistance levels.

Confidence Level: Medium

I'm not confident enough to make a bold buy or sell decision given the current market trends and Nvidia's recent stock price momentum. However, I believe that holding onto the stock for now could be a prudent decision, as it allows investors to ride out any potential volatility and potentially benefit from a positive earnings report.
